In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my-app-name").getOrCreate()
sc    = spark.sparkContext      # get the context

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
from operator import itemgetter

In [167]:
!wget https://s3-us-west-2.amazonaws.com/rithviksbucket/rac2015.csv

--2017-08-03 23:53:01--  https://s3-us-west-2.amazonaws.com/rithviksbucket/rac2015.csv
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 54.231.169.64
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|54.231.169.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169112328 (161M) [application/vnd.ms-excel]
Saving to: ‘rac2015.csv’

rac2015.csv         100%[===================>] 161.28M  64.2MB/s    in 2.5s    

2017-08-03 23:53:04 (64.2 MB/s) - ‘rac2015.csv’ saved [169112328/169112328]



In [168]:
#!unzip mort2015.csv.zip

In [169]:
#!head mort2015.csv

In [170]:
data = pd.read_csv("rac2015.csv")

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [171]:
len(data.columns)

20

In [172]:
record = []
for i in np.arange(1,21):
    x = 'record_' + str(i)
    record.append(x)
record

['record_1',
 'record_2',
 'record_3',
 'record_4',
 'record_5',
 'record_6',
 'record_7',
 'record_8',
 'record_9',
 'record_10',
 'record_11',
 'record_12',
 'record_13',
 'record_14',
 'record_15',
 'record_16',
 'record_17',
 'record_18',
 'record_19',
 'record_20']

In [173]:
cleanData = data[record]

In [174]:
cleanData.head()

,record_1,record_2,record_3,record_4,record_5,record_6,record_7,record_8,record_9,record_10,record_11,record_12,record_13,record_14,record_15,record_16,record_17,record_18,record_19,record_20
0,I500,L031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C349,F179,I251,I469,J440,J90,R042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,G309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,X91,S223,S299,T149,T71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,I250,S720,X590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
data_list = cleanData.values.tolist()
#data_list

In [176]:
cleanList = [ [y for y in x if str(y) != 'nan'] for x in data_list]
cleanList = [ list(np.unique(x)) for x in cleanList]

cleanList[4]

['I250', 'S720', 'X590']

In [177]:
for i in np.arange(0,30):
    print(cleanList[i])

['I500', 'L031']
['C349', 'F179', 'I251', 'I469', 'J440', 'J90', 'R042']
['G309']
['S223', 'S299', 'T149', 'T71', 'X91']
['I250', 'S720', 'X590']
['E780', 'F179', 'I499', 'I516']
['E274']
['I500']
['I350', 'I500']
['E039', 'F03', 'I694', 'R688', 'T142', 'W19']
['G20', 'I48', 'I64', 'J189']
['G309', 'I500']
['I219', 'I251', 'N19']
['R54', 'R688']
['C798', 'F179', 'K769']
['S099', 'X599']
['T71', 'X70']
['S015', 'S019', 'S069', 'X72']
['I64', 'R13']
['G20', 'I64', 'J189']
['I250', 'J449']
['E668', 'I500']
['A419', 'F179', 'J449']
['E86', 'G309', 'I469', 'R64']
['J189']
['F03']
['A409', 'I429', 'J154', 'J80', 'J969']
['I350', 'I469', 'I48', 'I500']
['I250']
['C80', 'F179', 'F199', 'I269', 'I279']


In [178]:
with open('test2015.txt', 'w') as file:
    file.writelines(' '.join(i) + '\n' for i in cleanList)

In [179]:
!hdfs dfs -put test2015.txt /user/hadoop/test2015.txt

In [180]:
from pyspark.mllib.fpm import FPGrowth
data = sc.textFile("test2015.txt")
transactions = data.map(lambda line: line.strip().split(' '))

In [181]:
model = FPGrowth.train(transactions, minSupport=0.0001, numPartitions=480)
result = model.freqItemsets().collect()

In [182]:
sorted(model.freqItemsets().collect(), key=itemgetter(1), reverse=True)

[FreqItemset(items=[u'I469'], freq=348777),
 FreqItemset(items=[u'I251'], freq=318853),
 FreqItemset(items=[u'I10'], freq=311972),
 FreqItemset(items=[u'F179'], freq=300456),
 FreqItemset(items=[u'I500'], freq=280560),
 FreqItemset(items=[u'J449'], freq=236848),
 FreqItemset(items=[u'F03'], freq=204006),
 FreqItemset(items=[u'A419'], freq=187077),
 FreqItemset(items=[u'J969'], freq=169705),
 FreqItemset(items=[u'C349'], freq=162036),
 FreqItemset(items=[u'I48'], freq=148778),
 FreqItemset(items=[u'I219'], freq=148208),
 FreqItemset(items=[u'J189'], freq=140579),
 FreqItemset(items=[u'G309'], freq=131807),
 FreqItemset(items=[u'E149'], freq=128082),
 FreqItemset(items=[u'I64'], freq=116168),
 FreqItemset(items=[u'J960'], freq=97586),
 FreqItemset(items=[u'J449', u'F179'], freq=94888),
 FreqItemset(items=[u'I250'], freq=84048),
 FreqItemset(items=[u'I10', u'I251'], freq=72178),
 FreqItemset(items=[u'R628'], freq=71807),
 FreqItemset(items=[u'C349', u'F179'], freq=71360),
 FreqItemset(ite

In [183]:
results2015 = pd.DataFrame(sorted(model.freqItemsets().collect(), key=itemgetter(1), reverse=True))

In [184]:
results2015.to_csv('results2015.txt',index=False, sep='\t')

In [17]:
#!head results.txt

[u'E142', u'J449']	2740
[u'R99', u'I10']	2738
[u'C029']	2735
[u'K746', u'I10']	2734
[u'E142', u'I48']	2734
[u'D649', u'N189']	2733
[u'D649', u'E149']	2727
[u'J189', u'A419', u'I500']	2725
[u'J459', u'I10']	2723
[u'R092', u'J449', u'F179']	2723


In [ ]:
sc.stop()